In [83]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime
import matplotlib.pyplot as plt
import collections
tqdm.pandas()

In [32]:
df = pd.read_csv('ratings.csv').drop('Unnamed: 0',axis=1)

# War of the Worlds (2005) 영화 두개 있는 것 하나로 합치기

In [7]:
df[df['title']=='War of the Worlds (2005)'].drop_duplicates('item') # 34048 item의 정보가 모든 면에서 64997 item 보다 디테일하다는 것을 알 수 있음 

,user,item,time,title,year,genre,director,writer
340117,11,34048,2009-01-01 05:41:29,War of the Worlds (2005),2005,"['Action', 'Adventure', 'Sci-Fi', 'Thriller']",['nm0000229'],"['nm0462895', 'nm0920229', 'nm0295264']"
1272022,11,64997,2009-08-25 03:20:46,War of the Worlds (2005),2005,"['Action', 'Sci-Fi']",NaN,['nm0920229']


In [8]:
df[df['title']=='War of the Worlds (2005)'].item.value_counts() # 영화 수도 34048이 훨씬 많음 -> 34048로 병합하자

34048    3022
64997     178
Name: item, dtype: int64

In [33]:
ind_64997 = df[df.item==64997].index
df.loc[ind_64997,'genre'] = df[df.item==34048]['genre'].iloc[0]
df.loc[ind_64997,'director'] = df[df.item==34048]['director'].iloc[0]
df.loc[ind_64997,'writer'] = df[df.item==34048]['writer'].iloc[0]
df.loc[ind_64997,'item'] = df[df.item==34048]['item'].iloc[0]

In [37]:
df.loc[ind_64997].head() # 기존 64997이었던 것들 다 34048처럼 바뀜

,user,item,time,title,year,genre,director,writer
1272022,11,34048,2009-08-25 03:20:46,War of the Worlds (2005),2005,"['Action', 'Adventure', 'Sci-Fi', 'Thriller']",['nm0000229'],"['nm0462895', 'nm0920229', 'nm0295264']"
1272023,572,34048,2011-12-20 02:56:13,War of the Worlds (2005),2005,"['Action', 'Adventure', 'Sci-Fi', 'Thriller']",['nm0000229'],"['nm0462895', 'nm0920229', 'nm0295264']"
1272024,2024,34048,2010-07-24 12:41:24,War of the Worlds (2005),2005,"['Action', 'Adventure', 'Sci-Fi', 'Thriller']",['nm0000229'],"['nm0462895', 'nm0920229', 'nm0295264']"
1272025,2070,34048,2010-07-07 13:05:20,War of the Worlds (2005),2005,"['Action', 'Adventure', 'Sci-Fi', 'Thriller']",['nm0000229'],"['nm0462895', 'nm0920229', 'nm0295264']"
1272026,5063,34048,2009-07-01 21:51:25,War of the Worlds (2005),2005,"['Action', 'Adventure', 'Sci-Fi', 'Thriller']",['nm0000229'],"['nm0462895', 'nm0920229', 'nm0295264']"


In [60]:
# 바꿔준 버전으로 다시 저장
df.to_csv('ratings2.csv')

# genre 세부 분석

In [47]:
genre = pd.read_csv("genres.tsv", delimiter='\t')

In [48]:
genre.genre.value_counts()

Drama          3490
Comedy         2408
Thriller       1466
Romance        1251
Action         1246
Crime           924
Adventure       891
Sci-Fi          653
Horror          602
Fantasy         557
Mystery         471
Children        432
Animation       355
War             346
Documentary     323
Musical         282
Western         139
Film-Noir        97
Name: genre, dtype: int64

In [55]:
# drama 대비 Advanture 수 비율
round(len(genre[genre['genre']=='Adventure']) / len(genre[genre['genre']=='Drama']) ,2)

0.26

In [56]:
# 전체 Action 영화 중 Adventure도 포함하는 영화 수 비율
len(df[df.genre.str.contains('Adventure')&df.genre.str.contains('Action')]) / len(df[df.genre.str.contains('Action')])

0.44070976050267724

In [57]:
# 전체 Action 영화 중 Drama도 포함하는 영화 수 비율
len(df[df.genre.str.contains('Drama')&df.genre.str.contains('Action')]) / len(df[df.genre.str.contains('Action')])

0.2943626893977149

### drama가 adventure보다 4배 정도 많음에도, Action 영화 장르인 것중에는 Drama 보다 Adventure가 1.5배 가량 많음
### 즉, genre를 원핫인코딩 하는 것보다는, embedding 시켜주는 것이 더 좋을 것이라 예상

In [65]:
df[df.genre=="['Adventure']"].drop_duplicates('item') # 이렇게 Adventure만 있는 영화들을 본 사람들은 Drama보다는 Action 영화 볼 확률이 더 높을 것이기 때문

,user,item,time,title,year,genre,director,writer
3827347,135,941,2006-08-04 10:26:18,"Mark of Zorro, The (1940)",1940,['Adventure'],['nm0541149'],"['nm0191547', 'nm0287124', 'nm0285218', 'nm058..."
4925226,775,5361,2005-04-18 06:42:16,White Fang (1991),1991,['Adventure'],['nm0459170'],['nm0742236']
5048611,1588,5357,2010-07-30 18:12:35,Iron Will (1994),1994,['Adventure'],NaN,"['nm0371088', 'nm0587429', 'nm0033578']"
5146127,5337,53919,2011-04-06 08:44:47,"Man in the Iron Mask, The (1939)",1939,['Adventure'],['nm0001843'],['nm0241416']


# 같은 제목(같은 영화인데 리메이크 한 것이라 가정) 영화면은 또 볼까?

### 12 Angry Men 이 영화는 1957년 영화인데 1997년에 리메이크한듯(감독은 다르지만 작가가 같음)

In [103]:
df[df.title=='12 Angry Men (1957)'].iloc[:1]

,user,item,time,title,year,genre,director,writer
2720766,58,1203,2006-04-03 10:25:22,12 Angry Men (1957),1957,['Drama'],['nm0001486'],['nm0741627']


In [104]:
df[df.title=='12 Angry Men (1997)'].iloc[:1]

,user,item,time,title,year,genre,director,writer
5104741,2511,77846,2013-01-20 19:20:12,12 Angry Men (1997),1997,"['Crime', 'Drama']",['nm0001243'],['nm0741627']


In [95]:
# 1957년작 본 사람 수
len(df[df.title=='12 Angry Men (1957)'])

5283

In [96]:
# 1997년 우리나라 성우 버전 영화 본 사람 수 
len(df[df.title=='12 Angry Men (1997)'])

234

In [97]:
# 위 두 영화 다 본 사람 수 
len(set(df[df.title=='12 Angry Men (1997)'].user).intersection(set(df[df.title=='12 Angry Men (1957)'].user)))

151

### 생각보다 영화 둘 다 본 사람 많다..(리메이크 영화 본 234 중 원작도 본 사람이 151명이나 됨) -> 영화 제목도 신경 써야 할듯

# 배트맨 같은 시리즈는 어떨까

In [112]:
batman = list(df[df.title.str.contains('Batman')].title.unique())
print(batman)

['Batman (1989)', 'Batman Forever (1995)', 'Batman Begins (2005)', 'Batman Returns (1992)', 'Batman/Superman Movie, The (1998)', 'Batman & Robin (1997)', 'Batman Beyond: Return of the Joker (2000)', 'Batman: The Dark Knight Returns, Part 2 (2013)', 'Batman: The Dark Knight Returns, Part 1 (2012)', 'Batman: Mask of the Phantasm (1993)', 'Batman: Gotham Knight (2008)', 'Batman: Under the Red Hood (2010)', 'Batman (1966)', 'Batman: Year One (2011)', 'Superman/Batman: Apocalypse (2010)']


In [129]:
# 이거 관련 진행하면서 dark knight 시리즈가 빠졌다는 것을 깨닫고 다크나이트 포함하고 다시 진행함
for i in df[(df.title.str.contains('Dark Knight'))&(~df.title.str.contains('Batman'))].title.unique():
    batman.append(i)
    print(i)

Dark Knight, The (2008)
Dark Knight Rises, The (2012)


In [130]:
batman_df = df[df.title.isin(batman)]

In [133]:
# 배트맨 관련 영화 본 사람 수 (전체 중 3분의 2 정도나 됨)
batman_df.user.nunique() / df.user.nunique()

0.6675382653061225

In [134]:
# 배트맨 영화 시청 순위 (총 17개)
batman_df.title.value_counts()

Batman Begins (2005)                              13405
Dark Knight, The (2008)                           13060
Batman (1989)                                      7130
Dark Knight Rises, The (2012)                      3337
Batman Returns (1992)                              2381
Batman Forever (1995)                              2062
Batman & Robin (1997)                               494
Batman: Mask of the Phantasm (1993)                 386
Batman (1966)                                       345
Batman: Under the Red Hood (2010)                   263
Batman: The Dark Knight Returns, Part 2 (2013)      260
Batman: The Dark Knight Returns, Part 1 (2012)      237
Batman: Gotham Knight (2008)                        230
Batman Beyond: Return of the Joker (2000)           214
Batman: Year One (2011)                             131
Batman/Superman Movie, The (1998)                    55
Superman/Batman: Apocalypse (2010)                   54
Name: title, dtype: int64

In [135]:
# 배트맨 영화들 여러개 본 사람들 (1개만 본 사람이 8284명, 17개 전부 다 챙겨서 본 사람이 1명 있음)
batman_df.groupby('user').count().item.value_counts()

1     8284
2     6635
3     3578
4     1380
5      594
6      245
7      109
8       46
11      19
9       17
10      16
12       5
13       4
17       1
14       1
Name: item, dtype: int64

### 시리즈 영화의 경우 2개 이상 봤으면 충분히 다른 배트맨 시리즈 인기 영화도 추천할만할듯
### (예를들어, 14개 챙겨본 사람은 나머지 3개 다 정답 10개 중 3개로 넣는게 좋을수도)

# 기타 시리즈 확인 (':'이 붙은 것들을 우선 시리즈물이라고 보기 - 좋은 방법은 아님)

In [18]:
# title들 unique한 list로
titles = list(df.title.unique())
# series는 dictionary로 (key는 series 이름, value에는 해당 시리즈 영화들)
series_dict = {}
for i in titles:
    if ':' in i:
        origin = i.split(':')[0]
        if origin not in series_dict:
            series_dict[origin] = [i]
        else:
            series_dict[origin].append(i)
# 시리즈들 dataframe화            
series_df = pd.DataFrame(list(series_dict.items()), columns=['series','titles'])            

In [27]:
series_df['counts'] = series_df.apply(lambda x : len(x.titles),axis=1)

In [31]:
series_df[series_df.counts != 1] # 하나만 있는 영화는 시리즈가 아니므로 제외 (중간에 이상한게 껴있기도 하고 ex. '3', 몇개 없기도 함 ex. harry potter)

,series,titles,counts
2,Ace Ventura,"[Ace Ventura: Pet Detective (1994), Ace Ventur...",2
5,Star Wars,"[Star Wars: Episode IV - A New Hope (1977), St...",7
8,X-Files,"[X-Files: I Want to Believe, The (2008), X-Fil...",2
11,X-Men,"[X-Men: The Last Stand (2006), X-Men: First Cl...",3
22,Lord of the Rings,[Lord of the Rings: The Fellowship of the Ring...,3
23,Mission,"[Mission: Impossible (1996), Mission: Impossib...",4
24,Kill Bill,"[Kill Bill: Vol. 1 (2003), Kill Bill: Vol. 2 (...",2
25,Pirates of the Caribbean,[Pirates of the Caribbean: The Curse of the Bl...,4
33,Chronicles of Narnia,"[Chronicles of Narnia: The Lion, the Witch and...",3
37,Transformers,"[Transformers: Revenge of the Fallen (2009), T...",4


In [36]:
# 위 방법으론 아래 해리포터 시리즈를 잡지 못함
df[df.title.str.contains('Harry Potter')].drop_duplicates('title').sort_values('year')

,user,item,time,title,year,genre,director,writer
1134506,11,4896,2009-01-02 01:19:59,Harry Potter and the Sorcerer's Stone (a.k.a. ...,2001,"['Adventure', 'Children', 'Fantasy']",['nm0001060'],"['nm0460141', 'nm0746830']"
689589,11,5816,2009-01-01 23:49:43,Harry Potter and the Chamber of Secrets (2002),2002,"['Adventure', 'Fantasy']",['nm0001060'],"['nm0460141', 'nm0746830']"
1115492,11,8368,2009-01-02 01:19:40,Harry Potter and the Prisoner of Azkaban (2004),2004,"['Adventure', 'Fantasy']",['nm0190859'],"['nm0460141', 'nm0746830']"
2125554,31,40815,2015-02-23 23:36:22,Harry Potter and the Goblet of Fire (2005),2005,"['Adventure', 'Fantasy', 'Thriller']",['nm0001565'],"['nm0460141', 'nm0746830']"
741388,11,54001,2009-01-01 23:53:17,Harry Potter and the Order of the Phoenix (2007),2007,"['Adventure', 'Drama', 'Fantasy']",['nm0946734'],"['nm0746830', 'nm0325533']"
2144521,31,69844,2015-02-23 23:39:08,Harry Potter and the Half-Blood Prince (2009),2009,"['Adventure', 'Fantasy', 'Mystery', 'Romance']",['nm0946734'],"['nm0460141', 'nm0746830']"
2110780,31,81834,2015-02-23 23:28:05,Harry Potter and the Deathly Hallows: Part 1 (...,2010,"['Action', 'Adventure', 'Fantasy']",['nm0946734'],"['nm0460141', 'nm0746830']"
2107669,31,88125,2015-02-23 23:27:55,Harry Potter and the Deathly Hallows: Part 2 (...,2011,"['Action', 'Adventure', 'Drama', 'Fantasy', 'M...",['nm0946734'],"['nm0460141', 'nm0746830']"


# 해리포터로 한번 더 실험

In [39]:
hp = list(df[df.title.str.contains('Harry Potter')].title.unique())

In [40]:
hp_df = df[df.title.isin(hp)]

In [41]:
# 해리포터 시리즈 영화 본 사람 수 (전체 중 3분의 1 정도 됨)
hp_df.user.nunique() / df.user.nunique()

0.37222576530612245

In [42]:
# 해리포터 영화 시청 순위 (총 8개)
hp_df.title.value_counts()

Harry Potter and the Prisoner of Azkaban (2004)                                                   7050
Harry Potter and the Goblet of Fire (2005)                                                        6494
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)    6431
Harry Potter and the Chamber of Secrets (2002)                                                    5706
Harry Potter and the Order of the Phoenix (2007)                                                  3942
Harry Potter and the Half-Blood Prince (2009)                                                     2801
Harry Potter and the Deathly Hallows: Part 2 (2011)                                               2710
Harry Potter and the Deathly Hallows: Part 1 (2010)                                               2609
Name: title, dtype: int64

### 여기서 보면 해리포터 시리즈 중 뒷 시리즈일 수록 순위가 떨어짐, 즉 만약 시리즈 추천 시 먼저 나온 영화를 우선하여 추천하는 방법이 좋을듯? (물론 시리즈 중 유독 순위가 높은 영화는 그거 먼저 추천해야겠지만.. ex) 배트맨 시리즈 중 다크나이트)

In [43]:
# 해리포터 영화들 여러개 본 사람들 (1개만 본 사람이 3347명, 8개 전부 다 챙겨서 본 사람이 764명 있음)
hp_df.groupby('user').count().item.value_counts()

1    3347
2    2027
3    1726
4    1592
5    1104
8     764
6     627
7     486
Name: item, dtype: int64

In [55]:
# 트와일라잇 역시 먼저 나온 영화일 수록 순위가 높음
df[df.title.str.contains('Twilight Saga')].title.value_counts()

Twilight Saga: New Moon, The (2009)                  279
Twilight Saga: Eclipse, The (2010)                   237
Twilight Saga: Breaking Dawn - Part 1, The (2011)    112
Twilight Saga: Breaking Dawn - Part 2, The (2012)     99
Name: title, dtype: int64

In [54]:
# 역시나 트와일라잇 1편을 제일 많이 봤다
df[df.title == 'Twilight (2008)'].user.nunique()

640

# 해리포터 시리즈 중 7개를 본 사람은 그렇다면 어떤 1개를 안 본 걸까?

In [60]:
hp_item_df = hp_df.groupby('user').count()[['item']]

In [66]:
# 해리포터 7개 본 user들
hp_7_user = hp_item_df[hp_item_df.item==7].index

In [76]:
hp_7_df = hp_df[hp_df.user.isin(hp_7_user)]

In [85]:
no_hp = collections.defaultdict(int)
for i in hp_7_user:
    one_hp = set(hp) - set(hp_7_df[hp_7_df.user==i].title)
    no_hp[str(one_hp)] += 1

In [88]:
# 7개의 해리포터 영화 본 사람 중 안본 영화들 순위 
sorted(no_hp.items(), key = lambda item: item[1], reverse = True)

[("{'Harry Potter and the Deathly Hallows: Part 2 (2011)'}", 159),
 ('{"Harry Potter and the Sorcerer\'s Stone (a.k.a. Harry Potter and the Philosopher\'s Stone) (2001)"}',
  60),
 ("{'Harry Potter and the Deathly Hallows: Part 1 (2010)'}", 56),
 ("{'Harry Potter and the Chamber of Secrets (2002)'}", 55),
 ("{'Harry Potter and the Half-Blood Prince (2009)'}", 47),
 ("{'Harry Potter and the Order of the Phoenix (2007)'}", 47),
 ("{'Harry Potter and the Prisoner of Azkaban (2004)'}", 36),
 ("{'Harry Potter and the Goblet of Fire (2005)'}", 26)]

### 위에 결과를 보면 해리포터 마지막 편인 7-2편을 안본 사람이 가장 많다. 이건 납득이 됨.
### 하지만, 7개의 해리포터 영화 봤고, 마지막 편까지 본사람이 중간에 한편만 안봤다는 건 납득이 잘 되지 않음
### 즉, 시리즈 물에서 다수의 작품을 봤음에도 중간에 한두편 정도를 안본 경우는 -> 해당 영화를 영화 추천 10개 중 포함시키는 것이 좋을 것 같다 (정답일 확률이 높음) ex. 해리포터 작품 8개 중 아즈카반의 죄수(3편)만 안봤을 경우(아즈카반의 죄수는 위에 보면 해리포터 시리즈 중 가장 인기 많았음) 아즈카반의 죄수가 정답 중 하나일 확률이 매우매우 높음

# 해리포터 작가

In [94]:
df[~df.writer.isnull()][df[~df.writer.isnull()].writer.str.contains('nm0460141')].title.unique()

array(['Harry Potter and the Chamber of Secrets (2002)',
       'Harry Potter and the Prisoner of Azkaban (2004)',
       "Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)",
       'Harry Potter and the Deathly Hallows: Part 2 (2011)',
       'Harry Potter and the Deathly Hallows: Part 1 (2010)',
       'Harry Potter and the Goblet of Fire (2005)',
       'Harry Potter and the Half-Blood Prince (2009)',
       'Wonder Boys (2000)', 'Amazing Spider-Man, The (2012)',
       'Fabulous Baker Boys, The (1989)'], dtype=object)

In [95]:
df[~df.writer.isnull()][df[~df.writer.isnull()].writer.str.contains('nm0746830')].title.unique()

array(['Harry Potter and the Chamber of Secrets (2002)',
       'Harry Potter and the Order of the Phoenix (2007)',
       'Harry Potter and the Prisoner of Azkaban (2004)',
       "Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)",
       'Harry Potter and the Deathly Hallows: Part 2 (2011)',
       'Harry Potter and the Deathly Hallows: Part 1 (2010)',
       'Harry Potter and the Goblet of Fire (2005)',
       'Harry Potter and the Half-Blood Prince (2009)'], dtype=object)

In [109]:
print(df[df.title=='Wonder Boys (2000)']['writer'].iloc[0])
print(df[df.title=='Amazing Spider-Man, The (2012)']['writer'].iloc[0])
print(df[df.title=='Fabulous Baker Boys, The (1989)']['writer'].iloc[0])

['nm0460141', 'nm0149290']
['nm0498278', 'nm0765091', 'nm0460141', 'nm0888743', 'nm0228492']
['nm0460141']


### nm0460141는 스티브 클로브스(해리포터 각본가), nm0746830는 jk롤링(해리포터 작가)  
### 스티브 클로브스의 작품수가 더 많아서 앞에 정렬되어 있음, 즉 단순히 작품 수 많은 director나 writer를 대표 한명으로 쓰기엔 어려움이 있음
### 공동으로 들어가 있는 수가 많을수록 뒤에 정렬하는 방식이나, 다른 방식으로 다시 정렬해야 할듯 (정렬 자체를 신경 안쓰거나)